# Introduction to atomman: Settings and Library

__Lucas M. Hale__, [lucas.hale@nist.gov](mailto:lucas.hale@nist.gov?Subject=ipr-demo), _Materials Science and Engineering Division, NIST_.
    
[Disclaimers](http://www.nist.gov/public_affairs/disclaimer.cfm) 

## 1. Introduction<a id='section1'></a>

*Added version 1.3.3*

The atomman package inherits Settings and Library from the [potentials package](https://github.com/usnistgov/potentials). The associated classes can be used to manage interactions with the NIST potentials reference library.

**Library Imports**

In [1]:
# Standard libraries
from pathlib import Path
import datetime       

# https://github.com/usnistgov/atomman
import atomman as am            

# Show atomman version
print('atomman version =', am.__version__)

# Show date of Notebook execution
print('Notebook executed on', datetime.date.today())

atomman version = 1.3.7
Notebook executed on 2021-02-19


## 2. Settings<a id='section2'></a>

The atomman.Settings class is a shortcut to the potentials.Settings class.  This allows for atomman, potentials, and iprPy to all share the same settings and local reference library.

In [2]:
settings = am.Settings()

# Show that atomman.Settings == potentials.Settings
print(repr(settings))

### 2.1. Set directory locations

- __directory__ is the location where the settings.json file is to be found.  The default directory is .NISTpotentials in the user's home directory.  This can be changed with Settings.set_directory(), but should only be done if multiple users/environments with different home directories are sharing settings.

- __library_directory__ is the location where the local library of reference records is located.  By default, this is inside the directory above.  It is recommended that it be set to somewhere more accessible if you wish to examine, modify or add to the reference records.

In [3]:
print('Settings directory is', settings.directory)
print('Library directory is', settings.library_directory)

Settings directory is C:\Users\lmh1\.NISTpotentials
Library directory is C:\Users\lmh1\Documents\library


In [4]:
# Change the settings directory - Not recommended
#settings.set_directory(<PATH>)

# Change the library directory - Recommended
#settings.set_library_directory(<PATH>)

print('Settings directory is', settings.directory)
print('Library directory is', settings.library_directory)

Settings directory is C:\Users\lmh1\.NISTpotentials
Library directory is C:\Users\lmh1\Documents\library


### 2.2. Set default database parameters

- __local__ indicates the default value for the local parameter to use when initializing a Library object (below), with a value of True indicating that the local library directory will be searched for matching records.

- __remote__ indicates the default value for the remote parameter to use when initializing a Library object (below), with a value of True indicating that the remote database https://potentials.nist.gov/ will be searched for matching records.

In [5]:
print('Default local parameter is', settings.local)
print('Default remote parameter is', settings.remote)

Default local parameter is True
Default remote parameter is True


In [6]:
# Change the local and remote parameters
#settings.set_local(True)
#settings.set_remote(False)

print('Default local parameter is', settings.local)
print('Default remote parameter is', settings.remote)

Default local parameter is True
Default remote parameter is True


## 3. Database interface <a id='section3'></a>

The atomman.library.Database class is a subclass of potentials.Database and extends it to provide support for crystal structure references in addition to the potentials references.  The class is used by atomman.load() for ['prototype'](1.4.10. prototype loading.ipynb) and ['crystal'](1.4.11. crystal loading.ipynb) styles and by [atomman.load_lammps_potential](2.1. Potential class.ipynb).  The class can also be directly accessed and used to explore the database for more involved investigations.

See the documentation notebooks for the [potentials package](https://github.com/usnistgov/potentials) for more details on interacting with the potentials records.

### 3.1. Initialize a Database object


Access parameters. Only change these if you are accessing an alternate database and/or have permission to add content.

- __host__ (*str, optional*) CDCS site to access.  Default value is 'https://potentials.nist.gov/'.
- __username__ (*str, optional*) User name to use to access the host site.  Default value of '' will access the site as an anonymous visitor.
- __password__ (*str, optional*) Password associated with the given username.  Not needed for anonymous access.
- __certification__ (*str, optional*) File path to certification file if needed for host.


Settings options. These allow the default settings from above to be overridden.

- __localpath__ (*str, optional*) Path to the local library directory to use.  If not given, will use the set library_directory setting.
- __local__ (*bool, optional*) Indicates if the load operations will check localpath for records. Default value is controlled by settings.
- __remote__ (*bool, optional*) Indicates if the load operations will download records from the remote database.  Default value is controlled by settings.  If a local copy exists, then setting this to False is considerably faster.

Other options

- __verbose__ (*bool, optional*) If True, info messages will be printed during operations.  Default value is False.
- __load__ (*bool, str or list, optional*) If True, citations, potentials and lammps_potentials will all be loaded during initialization. If False (default), none will be loaded.  Alternatively, a str or list can be given to specify which of the three record types to load.
- __status__ (*str, list or None, optional*) Only potential_LAMMPS records with the given status(es) will be loaded.  Allowed values are 'active' (default), 'superseded', and 'retracted'.  If None is given, then all potentials will be loaded.

In [7]:
# Load Database with default settings 
potdb = am.library.Database()
print('localpath =', potdb.localpath)
print('local =', potdb.local)
print('remote =', potdb.remote)

localpath = C:\Users\lmh1\Documents\library
local = True
remote = True


### 3.2. Exploring crystal prototypes

#### 3.2.1. Load crystal prototypes

The current prototypes in the local and/or remote database can be loaded into memory using the load_crystal_prototypes() method.

- __localpath__ (*str, optional*) Path to a local directory to check for records first.  If not given, will check localpath value set during object initialization.  If not given or set during initialization, then only the remote database will be loaded.
- __local__ (*bool, optional*) Indicates if records in localpath are to be loaded.  If not given, will use the local value set during initialization.
- __remote__ (*bool, optional*) Indicates if the records in the remote database are to be loaded. Setting this to be False is useful/faster if a local copy of the database exists.  If not given, will use the local value set during initialization.
- __verbose__ (*bool, optional*) If True, info messages will be printed during operations.  Default value is False.

In [8]:
potdb.load_crystal_prototypes(verbose=True)

Loaded 19 local crystal prototypes
Loaded 19 remote crystal prototypes
 - 0 new


#### 3.2.2. Get crystal prototype(s) 

The get_crystal_prototype() and get_crystal_prototypes() methods can be used to find one or all crystal prototypes that match the given parameters. 

Note that how the get methods work depends on if the associated records have been loaded.  If not loaded, then the get methods will query the remote database.  If loaded, then the get methods will search the set of loaded local+remote entries.  The two variations should, but are not guaranteed, to behave identically. 

- __id__ (*str or list, optional*) Prototype ID(s) to search for.  These are unique identifiers for each prototype based on comm.
- __key__ (*str or list, optional*) UUID4 key(s) to search for.  Each entry has a unique random-generated UUID4 key.
- __name__ (*str or list, optional*) Common name(s) to limit the search by.
- __prototype__ (*str or list, optional*) Prototype identifying composition(s) to limit the search by.
- __pearson__ (*str or list, optional*) The Pearson symbol(s) to limit the search by.
- __strukturbericht__ (*str or list, optional*) The strukturbericht identifier(s) to limit the search by.
- __sg_number__ (*int or list, optional*) The space group number(s) to limit the search by.
- __sg_hm__ (*str or list, optional*) The space group Hermann-Maguin identifier(s) to limit the search by.
- __sg_schoenflies__ (*str or list, optional*) The space group Schoenflies identifier(s) to limit the search by.
- __keyword__ (*str, optional*) If given, will limit the search to all records that contain the keyword substring.  Cannot be combined with any of the above parameters.
- __verbose__ (*bool, optional*) If True, info messages will be printed during operations.  Default value is False.



In [9]:
prototype = potdb.get_crystal_prototype(keyword='hcp')

Multiple matching crystal prototypes found.
1 A3'--alpha-La--double-hcp
2 A3--Mg--hcp
Select which one: 2


The returned CrystalPrototype object(s) have attributes associated with their metadata information as well as an atomman.System of the unit cell.

In [10]:
print(prototype.id)
print(prototype.ucell)

A3--Mg--hcp
avect =  [ 1.000,  0.000,  0.000]
bvect =  [-0.500,  0.866,  0.000]
cvect =  [ 0.000,  0.000,  1.633]
origin = [ 0.000,  0.000,  0.000]
natoms = 2
natypes = 1
symbols = (None,)
pbc = [ True  True  True]
per-atom properties = ['atype', 'pos']
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.000 |   0.000 |   0.000
      1 |       1 |   0.000 |   0.577 |   0.816


#### 3.2.3. download_crystal_prototypes

The download_crystal_prototypes method will download all entries in the remote to the local.

- __localpath__ (*path-like object, optional*) Path to a local directory where the files will be saved to.  If not given, will use the localpath value set during object initialization.
- __prototypes__ (*list of CrystalPrototype, optional*) A list of crystal prototypes to download. If not given, all prototypes will be downloaded.
- __format__ (*str, optional*) The file format to save the record files as.  Allowed values are 'xml' and 'json' (default).
- __indent__ (*int, optional*) The indentation spacing size to use for the locally saved record files. If not given, the JSON/XML content will be compact.
- __verbose__ (*bool, optional*) If True, info messages will be printed during operations.  Default value is False.

In [11]:
#potdb.download_crystal_prototypes(format='json', indent=4, verbose=True)

#### 3.2.4. crystal_prototypes and crystal_prototypes_df

Calling load_crystal_prototypes builds crystal_prototypes, a numpy array containing all CrystalPrototype objects, and crystal_prototypes_df, a pandas DataFrame of the prototype metadata.  Combined, these allow for more involved custom explorations of the loaded prototypes. 

In [12]:
# Display the metadata for all prototypes
potdb.crystal_prototypes_df

,key,id,name,prototype,pearson,strukturbericht,sg_number,sg_hm,sg_schoenflies,ucell,crystal_family,natypes,a,b,c,alpha,beta,gamma,natoms
0,d30980ad-ae18-425d-84cb-abf08577bdc8,A1--Cu--fcc,face-centered cubic,Cu,cF4,A1,225,F m -3 m,Oh^5,"avect = [ 1.000, 0.000, 0.000]\nbvect = [ ...",cubic,1,1.0,1.0,1.000000,90.0,90.0,90.0,4
2,f4101896-1e17-4736-a4d1-308b8934e8ce,A15--Cr3Si,beta-tungsten,Cr3Si,cP8,A15,223,P m -3 n,Oh^3,"avect = [ 1.000, 0.000, 0.000]\nbvect = [ ...",cubic,2,1.0,1.0,1.000000,90.0,90.0,90.0,8
1,e0126715-c7db-4d79-be80-707c572bebd6,A15--beta-W,beta-tungsten,b-W,cP8,A15,223,P m -3 n,Oh^3,"avect = [ 1.000, 0.000, 0.000]\nbvect = [ ...",cubic,1,1.0,1.0,1.000000,90.0,90.0,90.0,8
3,bc13827d-e1e6-4d70-8c3a-59399ad78b0f,A2--W--bcc,body-centered cubic,W,cI2,A2,229,I m -3 m,Oh^9,"avect = [ 1.000, 0.000, 0.000]\nbvect = [ ...",cubic,1,1.0,1.0,1.000000,90.0,90.0,90.0,2
4,154d1a2b-04da-4477-8962-a0ca0c17c8fc,A3'--alpha-La--double-hcp,double hcp,La,hP4,A3',194,P 6_3/m m c,D6h^4,"avect = [ 1.000, 0.000, 0.000]\nbvect = [-...",hexagonal,1,1.0,1.0,3.265986,90.0,90.0,120.0,4
5,4dab7fbb-108a-425e-8adb-d55de7b1134b,A3--Mg--hcp,hexagonal close-packed,Mg,hP2,A3,194,P 6_3/m m c,D6h^4,"avect = [ 1.000, 0.000, 0.000]\nbvect = [-...",hexagonal,1,1.0,1.0,1.632993,90.0,90.0,120.0,2
6,12654c83-c153-4b58-8805-f6dce43b7842,A4--C--dc,diamond cubic,C,cF8,A4,227,F d -3 m,Oh^7,"avect = [ 1.000, 0.000, 0.000]\nbvect = [ ...",cubic,1,1.0,1.0,1.000000,90.0,90.0,90.0,8
7,821526c5-561e-4f09-8a04-680419dc1495,A5--beta-Sn,white tin,b-Sn,tI4,A5,141,I 4_1/a m d,D4h^19,"avect = [ 1.000, 0.000, 0.000]\nbvect = [ ...",tetragonal,1,1.0,1.0,0.550000,90.0,90.0,90.0,4
8,76cbf2c1-fa86-41c4-85ae-c24f55c9f13a,A6--In--bct,body-centered tetragonal,In,tI2,A6,139,I 4/m m m,D4h^17,"avect = [ 1.000, 0.000, 0.000]\nbvect = [ ...",tetragonal,1,1.0,1.0,1.500000,90.0,90.0,90.0,2
9,c1e15cf8-6998-40ce-afbd-0b1cb6271a1f,A7--alpha-As,alpha As,a-As,hR6,A7,166,R -3 m,D3d^5,"avect = [ 1.000, 0.000, 0.000]\nbvect = [-...",hexagonal,1,1.0,1.0,2.777000,90.0,90.0,120.0,6


In [13]:
# Find the listings in df that are tetragonal
is_tetragonal = potdb.crystal_prototypes_df.crystal_family == 'tetragonal'

# Use is_tetragonal mask to retrieve the associated prototypes
for prototype in potdb.crystal_prototypes[is_tetragonal]:
    print(prototype.id)

A5--beta-Sn
A6--In--bct
L1_0--AuCu


### 3.3. Exploring relaxed crystals

The Database object has built-in methods for exploring the relaxed crystals in a similar manner as the crystal prototypes:

- load_relaxed_crystals() builds listings for all relaxed crystals from local and/or remote
- relaxed_crystals and relaxed_crystals_df are the listings built by load_relaxed_crystals()
- get_relaxed_crystals() and get_relaxed_crystal() retrieve matching entries
- download_relaxed_crystals() downloads all remote entries to local

__HOWEVER__, many of these operations are incredibly slow due to the large number of relaxed crystal entries.  There are two suggested methods to best reduce the processing time, which are listed below.

#### 3.3.1. Remote-only searches

The simplest method is to call get_relaxed_crystals() or get_relaxed_crystal() without calling load_relaxed_crystals().  This will send queries to the remote database to retrieve matching records.  Giving parameters that limit the search to as few entries as possible will reduce the amount of information that needs to be downloaded. 

Limitations: Requires internet access, no user-defined local entries will be included, and each search involves a separate internet request.

Parameters for get_relaxed_crystals and get_relaxed_crystal

- __key__ (*str or list, optional*) UUID4 key(s) to search for.  Each entry has a unique random-generated UUID4 key.
- __method__ (*str or list or None, optional*) The relaxation method used.  Allowed values are dynamic, static and box. Default value is dynamic (the most rigorous relaxation method).  All will be loaded if set to None.
- __standing__ (*str or list or None, optional*) "good" records are the unique crystals found with the most rigorous relaxation, and with known prototypes over DFT structures.  "bad" are records filtered out, usually for being duplicates.  Default value is "good".  All will be loaded if set to None.
- __family__ (*str or atomman.library.CrystalPrototype or list, optional*) The crystal family associated with the relaxed crystal - either crystal prototype name or MP/OQMD database entry name.
- __parent_key__ (*str or list, optional*) The UUID4 key(s) assigned to the calculation that the record is based on.
- __potential__ (*atomman.lammps.Potential or list, optional*) A loaded LAMMPS potential object to limit the search by.
- __potential_LAMMPS_id__ (*str or list, optional*) The id for a LAMMPS implemented potential to limit the search by.
- __potential_LAMMPS_key__ (*str or list, optional*) The UUID4 for a LAMMPS implemented potential to limit the search by.
- __potential_id__ (*str or list, optional*) The id for a potential to limit the search by.
- __potential_key__ (*str or list, optional*) The UUID4 for a potential to limit the search by.
- __symbols__ (*str or list, optional*) Element symbols in the crystal to limit the search by.
- __natypes__ (*int or list, optional*) The number of unique element model symbols in the crystal to limit the search by.
- __natoms__ (*int or list, optional*) The number of unique atoms in the crystal's unit cell to limit the search by.
- __keyword__ (*str, optional*) If given, will limit the search to all records that contain the keyword substring.  Cannot be combined with any of the above parameters.
- __verbose__ (*bool, optional*) If True, info messages will be printed during operations.  Default value is False.

In [14]:
# Fetch all records for fcc gold
fcc_gold = potdb.get_relaxed_crystals(family='A1--Cu--fcc', symbols=['Au'])

# List the matches, the a lattice constant and the associated potential
print('alat    Ecoh   potential_LAMMPS_id')
for crystal in fcc_gold:
    print(f'{crystal.ucell.box.a:.4f} {crystal.cohesive_energy:.4f} {crystal.potential_LAMMPS_id}')

alat    Ecoh   potential_LAMMPS_id
4.0685 -4.2994 2012--Norman-G-E--Au--LAMMPS--ipr1
4.0801 -3.9300 2004--Zhou-X-W--Cu-Ag-Au--LAMMPS--ipr2
4.0801 -3.9300 2004--Zhou-X-W--Au--LAMMPS--ipr2
4.0801 -3.9300 2004--Zhou-X-W--Cu-Ag-Au--LAMMPS--ipr1
4.0798 -3.9300 2004--Zhou-X-W--Au--LAMMPS--ipr1
4.0800 -3.9300 1986--Foiles-S-M--Au--LAMMPS--ipr1
4.0800 -3.9300 1986--Foiles-S-M--Ag-Au-Cu-Ni-Pd-Pt--LAMMPS--ipr1
4.0780 -3.9300 2017--Purja-Pun-G-P--Au--LAMMPS--ipr1
4.0800 -3.9300 1989--Adams-J-B--Au--LAMMPS--ipr1
4.0800 -3.9300 1989--Adams-J-B--Ag-Au-Cu-Ni-Pd-Pt--LAMMPS--ipr1
4.0701 -3.9242 2005--Grochola-G--Au--LAMMPS--ipr1
4.1820 -3.9024 2018--Starikov-S-V--Si-Au--LAMMPS--ipr1
4.1820 -3.9024 2018--Starikov-S-V--Si-Au--LAMMPS--ipr2
4.0800 -3.8100 2010--Olsson-P-A-T--Au--LAMMPS--ipr1
4.0780 -3.7890 1987--Ackland-G-J--Au--LAMMPS--ipr2
4.0780 -3.7890 1987--Ackland-G-J--Au--LAMMPS--ipr1
4.1529 -3.0332 2017--OBrien-C-J--Pt-Au--LAMMPS--ipr1
6.2107 -0.6133 2012--Norman-G-E--Au--LAMMPS--ipr1


#### 3.3.2. Local-only searches

The other recommended method is to load only records from the local and then perform the get searches. Doing this, of course, requires that any relaxed crystal records first be downloaded to the local library directory, which can be done one of two ways

1. First performing a remote-only search as mentioned above, then passing the list of matching records as parameters to the download_relaxed_crystals() method.  This allows for records of interest to be saved for accessing later.

2. If you want all available entries in the local copy, download or clone the git repository at https://github.com/lmhale99/potentials-library to the local library directory.  While all the entries can be downloaded using download_relaxed_crystals(), doing so is incredibly slow.

Either way, once the crystal records are in the local directory they can be loaded with load_relaxed_crystals(remote=False), and then the get_relaxed_crystals() and get_relaxed_crystal() methods will work without needing internet access. 


In [15]:
potdb.load_relaxed_crystals(remote=False, verbose=True)

Loaded 59326 local relaxed crystals


In [16]:
# Fetch all records for fcc gold
fcc_gold = potdb.get_relaxed_crystals(family='A1--Cu--fcc', symbols=['Au'])

# List the matches, the a lattice constant and the associated potential
print('alat    Ecoh   potential_LAMMPS_id')
for crystal in fcc_gold:
    print(f'{crystal.ucell.box.a:.4f} {crystal.cohesive_energy:.4f} {crystal.potential_LAMMPS_id}')

alat    Ecoh   potential_LAMMPS_id
4.0800 -3.9300 1989--Adams-J-B--Ag-Au-Cu-Ni-Pd-Pt--LAMMPS--ipr1
4.0800 -3.8100 2010--Olsson-P-A-T--Au--LAMMPS--ipr1
7.8429 -0.8357 2020--Starikov-S--Si-Au-Al--LAMMPS--ipr1
4.0701 -3.9242 2005--Grochola-G--Au--LAMMPS--ipr1
4.0800 -3.9300 1986--Foiles-S-M--Au--LAMMPS--ipr1
4.1820 -3.9024 2018--Starikov-S-V--Si-Au--LAMMPS--ipr1
4.0685 -4.2994 2012--Norman-G-E--Au--LAMMPS--ipr1
4.1499 -3.8612 2020--Starikov-S--Si-Au-Al--LAMMPS--ipr1
4.0780 -3.9300 2017--Purja-Pun-G-P--Au--LAMMPS--ipr1
4.1529 -3.0332 2017--OBrien-C-J--Pt-Au--LAMMPS--ipr1
4.0801 -3.9300 2004--Zhou-X-W--Cu-Ag-Au--LAMMPS--ipr2
4.0798 -3.9300 2004--Zhou-X-W--Au--LAMMPS--ipr1
4.0800 -3.9300 1986--Foiles-S-M--Ag-Au-Cu-Ni-Pd-Pt--LAMMPS--ipr1
4.0780 -3.7890 1987--Ackland-G-J--Au--LAMMPS--ipr1
4.0780 -3.7890 1987--Ackland-G-J--Au--LAMMPS--ipr2
4.0801 -3.9300 2004--Zhou-X-W--Au--LAMMPS--ipr2
4.0800 -3.9300 1989--Adams-J-B--Au--LAMMPS--ipr1
6.1962 -1.4338 2020--Starikov-S--Si-Au-Al--LAMMPS--ipr1
6.21

### 3.4. Exploring potentials

The Database object allows for searching potential entries by citation and searching lammps implementations of the potentials.

Methods and attributes for exploring potential listings:

- **widget_search_potentials** provides a Jupyter GUI for exploring potential entries.
- **potentials** is a list of the entries as objects.  This is generated if load_potentials() is called.
- **potentials_df** is a pandas.DataFrame containing the content of the listings.  This is generated if load_potentials() is called.
- **load_potentials()** builds potentials and potentials_df based on entries found in the local and/or remote databases.
- **get_potentials() and get_potential()** retrieves all potentials or one potential, respectively, that match with any given parameters. 
- **download_potentials** will download all potential entries from remote to local.  
- **save_potentials** saves user-defined entries to local.
- **upload_potential** uploads a new/updated potential listing to remote.  Requires write permission for the remote database.
- **delete_potential** will delete a listing from local and/or remote (if permissions are allowed).

Methods and attributes for exploring LAMMPS potential listings:

- **widget_lammps_potential** provides a Jupyter GUI for exploring LAMMPS potential entries and retrieving any associated parameter files.
- **lammps_potentials** is a list of the entries as objects.  This is generated if load_lammps_potentials() is called.
- **lammps_potentials_df** is a pandas.DataFrame containing the content of the listings.  This is generated if load_lammps_potentials() is called.
- **load_lammps_potentials** builds lammps_potentials and lammps_potentials_df based on entries found in the local and/or remote databases.
- **get_lammps_potentials and get_lammps_potential** retrieves all LAMMPS potentials or one LAMMPS potential, respectively, that match with any given parameters. 
- **download_lammps_potentials** will download all LAMMPS potential entries from remote to local.
- **get_lammps_potentials_files** will download any parameter files associated with one or more LAMMPS potentials to local.
- **save_lammps_potentials** saves user-defined entries to local.
- **upload_lammps_potential** uploads a new/updated LAMMPS potential listing to remote.  Requires write permission for the remote database.
- **delete_lammps_potential** will delete a listing from local and/or remote (if permissions are allowed).

See the documentation notebooks for the [potentials package](https://github.com/usnistgov/potentials) for more details on interacting with the potentials records.